# Model Evaluation Report

In [ ]:
train_dir = None
checkpoint_path = None
string_lookup_config_path = None

n_molecules = None
subset_size = None

In [ ]:
from pathlib import Path

from IPython.display import Markdown, display

from mol_gen.molecule_generation.molecule_generator import MoleculeGenerator
from mol_gen.training.evaluate import (calculate_percent_selfies_novel,
                                       calculate_percent_selfies_valid,
                                       calculate_percent_unique,
                                       draw_subset_selfies,
                                       plot_descriptor_distributions)

In [ ]:
mol_generator = MoleculeGenerator.from_files(checkpoint_path, string_lookup_config_path)

display(Markdown(f"## {Path(checkpoint_path).name}"))
mols = mol_generator.generate_molecules(n_molecules)

display(Markdown("### Duplicates"))
percent_unique = calculate_percent_unique(mols)
print(f"{percent_unique}% of molecules generated unique.")

display(Markdown("### Novelty"))
percent_novel = calculate_percent_selfies_novel(mols, train_dir)
print(f"{percent_novel}% of molecules generated novel.")

display(Markdown("### Validity"))
percent_valid = calculate_percent_selfies_valid(mols)
print(f"{percent_valid}% of molecules generated valid.")

display(Markdown("### Properties Distribution"))
plot_descriptor_distributions(mols)

display(Markdown("### Subset"))
display(draw_subset_selfies(mols, subset_size))